# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

# Задание

1.Загрузите датасет. Описание датасета можно посмотреть здесь - http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/.

2.Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [33]:
import pandas as pd

In [34]:
data = pd.read_csv('SMSSpamCollection.txt', sep = '\t', names = ["label", "text"], header = None)

In [29]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3.Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [35]:
data = data.replace(to_replace=['ham', 'spam'], value=[0, 1])

In [31]:
# data.label = [1 if x == 'spam' else 0 for x in data.label]

In [37]:
data.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [38]:
labels = data.label
texts = data.text

4.Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
X = CountVectorizer(texts)
y = labels

5.Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [83]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
import numpy as np

In [84]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [92]:
np.random.seed(2)
answer1 = cross_val_score(text_classifier(CountVectorizer(), LogisticRegression()), texts, labels, cv = 10, scoring = 'f1').mean()

In [114]:
answer1

0.9326402983610631

In [94]:
def write_answer_1(answer):
    with open("sms_answer_1.txt", "w") as fout:
        fout.write(str(answer))

In [95]:
write_answer_1(round(answer1, 1))

6.А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [91]:
clf_pipeline = Pipeline(
            [("vectorizer", CountVectorizer()),
            ("classifier", LogisticRegression())]
        )


clf_pipeline.fit(texts, labels)

print clf_pipeline

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [99]:
print clf_pipeline.predict(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"])

[1]


In [100]:
answer2 = clf_pipeline.predict(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", 
                                "FreeMsg: Txt: claim your reward of 3 hours talk time",
                                "Have you visited the last lecture on physics?",
                                "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                                "Only 99$"])

In [109]:
print " ".join([str(i) for i in answer2])

1 1 0 0 0


In [117]:
def write_answer_2(answer):
    with open("sms_answer_2.txt", "w") as fout:
        fout.write(str(answer))

In [111]:
write_answer_2(" ".join([str(i) for i in answer2]))

7.Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [112]:
answer3_1 = cross_val_score(text_classifier(CountVectorizer(ngram_range=(2,2)), LogisticRegression()), texts, labels, cv = 10, scoring = 'f1').mean()
answer3_2 = cross_val_score(text_classifier(CountVectorizer(ngram_range=(3,3)), LogisticRegression()), texts, labels, cv = 10, scoring = 'f1').mean()
answer3_3 = cross_val_score(text_classifier(CountVectorizer(ngram_range=(1,3)), LogisticRegression()), texts, labels, cv = 10, scoring = 'f1').mean()

In [113]:
print answer3_1, answer3_2, answer3_3

0.822422066419 0.725016155547 0.925138255865


In [118]:
def write_answer_3(answer):
    with open("sms_answer_3.txt", "w") as fout:
        fout.write(str(answer))

In [120]:
write_answer_3(" ".join([str(round(answer3_1, 2)), str(round(answer3_2, 2)), str(round(answer3_3, 2))]))

8.Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор. 

In [123]:
from sklearn.naive_bayes import MultinomialNB

In [128]:
count_vectorizer = CountVectorizer(ngram_range=(2,2))
X = count_vectorizer.fit_transform(texts)
answer4_1 = cross_val_score(MultinomialNB(), X, labels, cv = 10, scoring = 'f1').mean()

In [136]:
count_vectorizer = CountVectorizer(ngram_range=(3,3))
X = count_vectorizer.fit_transform(texts)
answer4_2 = cross_val_score(MultinomialNB(), X, labels, cv = 10, scoring = 'f1').mean()

In [138]:
count_vectorizer = CountVectorizer(ngram_range=(1,3))
X = count_vectorizer.fit_transform(texts)
answer4_3 = cross_val_score(MultinomialNB(), X, labels, cv = 10, scoring = 'f1').mean()

In [140]:
print answer4_1, answer4_2, answer4_3

0.645501517799 0.378719485246 0.888485965606


In [141]:
def write_answer_4(answer):
    with open("sms_answer_4.txt", "w") as fout:
        fout.write(str(answer))

In [142]:
write_answer_4(" ".join([str(round(answer4_1, 2)), str(round(answer4_2, 2)), str(round(answer4_3, 2))]))

9.Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [144]:
score = cross_val_score(text_classifier(TfidfVectorizer(), LogisticRegression()), texts, labels, cv = 10, scoring = 'f1').mean()

In [145]:
print "CountVectorizer: {},  TfidfVectorizer: {}".format(answer1, score)

CountVectorizer: 0.932640298361,  TfidfVectorizer: 0.878510045534


In [146]:
def write_answer_5(answer):
    with open("sms_answer_5.txt", "w") as fout:
        fout.write(str(answer))

In [147]:
write_answer_5("-1")